In [1]:
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt
import re
import html5lib

3.5.2 |Anaconda 4.1.1 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
1.11.1
0.18.1


In [2]:
# df will be our set with all fields, it originated by searching the foodfacts website for US products
# we're going to work with Nikki's csv, which includes her calories_100g field

df = pd.read_csv("groomed_food_facts_data.cs", sep = '\t')

# and also include her adjustment for the fields that were truncating

pd.set_option('display.max_colwidth', -1)
df.head()

,Unnamed: 0,code,url,creator,created_t,last_modified_t,product_name,generic_name,quantity,packaging,...,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,calories_100g
0,0,5010092093045,http://world.openfoodfacts.org/product/5010092093045,bcatelin,1389309305,1461479010,Soft white,White bread,800g,Plastic bag,...,NaN,NaN,NaN,NaN,NaN,NaN,125.0,-1.0,-1.0,232.400
1,1,44000030377,http://world.openfoodfacts.org/product/0044000030377,openfoodfacts-contributors,1385850411,1459174448,Wheat Thins Original,NaN,258g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,7832309,http://world.openfoodfacts.org/product/07832309,openfoodfacts-contributors,1403210081,1458995984,Diet Dr Pepper,NaN,NaN,can,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5099353000169,http://world.openfoodfacts.org/product/5099353000169,bcatelin,1385926289,1413659845,Eggs,Eggs,6,Cardbox,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,82592720153,http://world.openfoodfacts.org/product/0082592720153,openfoodfacts-contributors,1389308826,1459174499,Green Machine,NaN,15.2 fl. oz (450 mL),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.532


In [3]:
###verify the nutrition facts add up with the manufacturer's data
### it appears from the site's documentation that 
###http://world.openfoodfacts.org/data/data-fields.txt
###[nutrient]_100gr is nutrients per 100gr or mL
###step 1, grab data for an identifable product

target_list = ['code', 'url', 'product_name', 'energy_100g','carbohydrates_100g', 
    'fat_100g', 'proteins_100g','serving_size']

df[target_list].iloc[20]

code                  36632036506                                         
url                   http://world.openfoodfacts.org/product/0036632036506
product_name          Activia light blueberry                             
energy_100g           222                                                 
carbohydrates_100g    9.73                                                
fat_100g              0                                                   
proteins_100g         3.54                                                
serving_size          1 container (113g)                                  
Name: 20, dtype: object

In [7]:
#step 2: verify the data on the site's own product page
import html5lib
site = 'http://www.activia.us.com/probiotic-yogurt/products/activia-light-blueberry'
blueberry_yougurt = pd.read_html(site)[0].set_index('Nutritional Facts')
blueberry_yougurt

,per serving (113g),Calories from Fat
Nutritional Facts,,
NaN,NaN,% Daily Value*
Calories,60,0
Total Fat,0g,0%
Saturated Fat,0g,0%
Trans Fat,0g,NaN
Cholesterol,<5mg,1%
Sodium,65mg,3%
Potassium,200mg,6%
Total Carbohydrate,11g,4%


In [8]:
### the calroies(enerygy) does not look like it is 220 cals/100gr, but the carb/protein/fat ratio looks about right
### verify the macronutrients per 100gr are equivalent to the food facts db:
print(float(blueberry_yougurt.loc['Protein'][0].strip('g'))/1.13)
print(float(blueberry_yougurt.loc['Total Carbohydrate'][0].strip('g'))/1.13)
print(float(blueberry_yougurt.loc['Total Fat'][0].strip('g'))/1.13)

3.5398230088495577
9.734513274336283
0.0


In [9]:
###the total macronutrients per 100 grams posted in the food facts db roughly matches what is posted on the product's
###own site.  Let's check anoother product, this time at random:
np.random.seed(4)
df[target_list].iloc[np.random.randint(0,len(df))]

code                  27000379097                                         
url                   http://world.openfoodfacts.org/product/0027000379097
product_name          Crushed tomatoes                                    
energy_100g           155                                                 
carbohydrates_100g    7.44                                                
fat_100g              0                                                   
proteins_100g         1.65                                                
serving_size          121 g                                               
Name: 1146, dtype: object

In [10]:
site = 'http://www.hunts.com/nutrition-label?upc=2700037909&inline=false'
hunts_tomatoes = pd.read_html(site)
hunts_tomatoes[1]

,0,1,2
0,Nutrition Facts,NaN,NaN
1,Serving Size 1/2 cup (121g),NaN,NaN
2,Servings Per Container about 7,NaN,NaN
3,Amount Per Serving,NaN,NaN
4,Calories 45Calories from Fat 0,NaN,NaN
5,NaN,% Daily Value*,NaN
6,Total Fat 0g,0%,NaN
7,NaN,Saturated Fat 0g,0%
8,NaN,Trans Fat 0g,NaN
9,Cholesterol 0mg,0%,NaN


In [11]:
# let's see how many of our rows have data in these target categories.  
# if we screen out those that are missing target data, how many items will we be left with?

for item in target_list:
    print(item, ":", df[item].count())


code : 2820
url : 2820
product_name : 2356
energy_100g : 1283
carbohydrates_100g : 1292
fat_100g : 1295
proteins_100g : 1284
serving_size : 1362


In [12]:
df2 = df[target_list]
df2.head()

,code,url,product_name,energy_100g,carbohydrates_100g,fat_100g,proteins_100g,serving_size
0,5010092093045,http://world.openfoodfacts.org/product/5010092093045,Soft white,1007.0,44.6,2.0,9.000,50g
1,44000030377,http://world.openfoodfacts.org/product/0044000030377,Wheat Thins Original,NaN,NaN,NaN,NaN,NaN
2,7832309,http://world.openfoodfacts.org/product/07832309,Diet Dr Pepper,NaN,NaN,NaN,NaN,NaN
3,5099353000169,http://world.openfoodfacts.org/product/5099353000169,Eggs,NaN,NaN,NaN,NaN,NaN
4,82592720153,http://world.openfoodfacts.org/product/0082592720153,Green Machine,244.0,13.8,0.0,0.833,8 fl oz (240 mL)


In [13]:
# have a look at some of the rows

df2.iloc[580:605]

,code,url,product_name,energy_100g,carbohydrates_100g,fat_100g,proteins_100g,serving_size
580,12000130274,http://world.openfoodfacts.org/product/0012000130274,NaN,195.0,12.3,0.00,0.0,1 Bottle (20 fl oz) (591 mL)
581,810165016415,http://world.openfoodfacts.org/product/0810165016415,NaN,NaN,NaN,NaN,NaN,NaN
582,856820160048,http://world.openfoodfacts.org/product/0856820160048,NaN,NaN,NaN,NaN,NaN,NaN
583,617237641564,http://world.openfoodfacts.org/product/0617237641564,NaN,NaN,NaN,NaN,NaN,NaN
584,8000500026731,http://world.openfoodfacts.org/product/8000500026731,NaN,NaN,NaN,NaN,NaN,NaN
585,38000576089,http://world.openfoodfacts.org/product/0038000576089,NaN,NaN,NaN,NaN,NaN,NaN
586,8000500205167,http://world.openfoodfacts.org/product/8000500205167,NaN,NaN,NaN,NaN,NaN,NaN
587,70896732057,http://world.openfoodfacts.org/product/0070896732057,NaN,NaN,NaN,NaN,NaN,NaN
588,799857655371,http://world.openfoodfacts.org/product/0799857655371,NaN,NaN,NaN,NaN,NaN,NaN
589,810675000225,http://world.openfoodfacts.org/product/0810675000225,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# looks like some of these rows may be poor quality (many NaN fields), example row 581

df2.iloc[581]

code                  810165016415                                        
url                   http://world.openfoodfacts.org/product/0810165016415
product_name          NaN                                                 
energy_100g           NaN                                                 
carbohydrates_100g    NaN                                                 
fat_100g              NaN                                                 
proteins_100g         NaN                                                 
serving_size          NaN                                                 
Name: 581, dtype: object

In [15]:
# use notnull to eliminate the rows that inluce NaNs in the fields we wish to study.

clean_df2 = df2[pd.notnull(df['calories_100g'])]
clean_df2.head()

,code,url,product_name,energy_100g,carbohydrates_100g,fat_100g,proteins_100g,serving_size
0,5010092093045,http://world.openfoodfacts.org/product/5010092093045,Soft white,1007.0,44.60,2.00,9.000,50g
4,82592720153,http://world.openfoodfacts.org/product/0082592720153,Green Machine,244.0,13.80,0.00,0.833,8 fl oz (240 mL)
6,9800895250,http://world.openfoodfacts.org/product/0009800895250,Nutella,2260.0,56.80,32.40,5.410,2 tbsp (37 g)
16,71443003903,http://world.openfoodfacts.org/product/0071443003903,NaN,1590.0,76.20,9.52,2.380,42 g
20,36632036506,http://world.openfoodfacts.org/product/0036632036506,Activia light blueberry,222.0,9.73,0.00,3.540,1 container (113g)


In [16]:
# this new df should be pretty balanced, not so many NaNs left.  let's see.
# note that I generic_name was added back as some of the product_name fields are unhelpful (ie "soft white" - ?)
# it's okay then that this field is a bit underpopulated, we'll use it just to fill in product name when unhelpful

clean_df2.count()

code                  1270
url                   1270
product_name          1223
energy_100g           1251
carbohydrates_100g    1270
fat_100g              1270
proteins_100g         1270
serving_size          1215
dtype: int64

In [31]:
# at this point we are well positioned to be able to analyse calorie count for any subset for our data.
# now what about the other nutrients: does a "basket" provide adequate nutrition, per IOM RDAs?
# let's go back to our original data set and see what nutrient data is available

# use this to search to compare to the nutrients studied by the IOM
# we're going to search on columns that include _100g, many of which are nutrients
# then need to work the formatting, removing _100g, replacing the "-" in leading position and inside the string

column_list = list(df.columns.values)

products_nutrients_list = []
for column in column_list:
    if re.search('_100g$', column):  # this gives us all the columns that include _100g, many of which are nutrients
        if re.search('^-', column):
            products_nutrients_list.append((column.replace("-", " "))[1:-5])  
        else:
            products_nutrients_list.append((column.replace("-", " "))[0:-5])
            
print(products_nutrients_list)

['energy', 'energy from fat', 'fat', 'saturated fat', 'butyric acid', 'caproic acid', 'caprylic acid', 'capric acid', 'lauric acid', 'myristic acid', 'palmitic acid', 'stearic acid', 'arachidic acid', 'behenic acid', 'lignoceric acid', 'cerotic acid', 'montanic acid', 'melissic acid', 'monounsaturated fat', 'polyunsaturated fat', 'omega 3 fat', 'alpha linolenic acid', 'eicosapentaenoic acid', 'docosahexaenoic acid', 'omega 6 fat', 'linoleic acid', 'arachidonic acid', 'gamma linolenic acid', 'dihomo gamma linolenic acid', 'omega 9 fat', 'oleic acid', 'elaidic acid', 'gondoic acid', 'mead acid', 'erucic acid', 'nervonic acid', 'trans fat', 'cholesterol', 'carbohydrates', 'sugars', 'sucrose', 'glucose', 'fructose', 'lactose', 'maltose', 'maltodextrins', 'starch', 'polyols', 'fiber', 'proteins', 'casein', 'serum proteins', 'nucleotides', 'salt', 'sodium', 'alcohol', 'vitamin a', 'beta carotene', 'vitamin d', 'vitamin e', 'vitamin k', 'vitamin c', 'vitamin b1', 'vitamin b2', 'vitamin pp', '

In [37]:
# pull in data from https://www.consumerlab.com/RDAs/ that shows RDAs (DRIs) from the IOM
# what are the nutrients for which recommendations are available?

nutrients = pd.read_html("https://www.consumerlab.com/RDAs/")[6]
nutrients


,0,1,2,3,4,5
0,Vitamins,B Vitamins,B Vitamins (cont'd),Minerals,Minerals(cont'd),Minerals(cont'd)
1,Vitamin A,Biotin,Riboflavin,Calcium,Iron,Potassium
2,Vitamin C,Choline,Thiamin,Chromium,Magnesium,Selenium
3,Vitamin D,Folate,Vitamin B-6,Copper,Molybdenum,Zinc
4,Vitamin E,Niacin,Vitamin B-12,Fluoride,Manganese,NaN
5,Vitamin K,Pantothenic Acid,NaN,Iodine,Phosphorus,NaN


In [38]:
# we don't need that first row

nutrients2 = nutrients.drop(0)
nutrients2

,0,1,2,3,4,5
1,Vitamin A,Biotin,Riboflavin,Calcium,Iron,Potassium
2,Vitamin C,Choline,Thiamin,Chromium,Magnesium,Selenium
3,Vitamin D,Folate,Vitamin B-6,Copper,Molybdenum,Zinc
4,Vitamin E,Niacin,Vitamin B-12,Fluoride,Manganese,NaN
5,Vitamin K,Pantothenic Acid,NaN,Iodine,Phosphorus,NaN


In [78]:
# let's put these in a list
iom_list = nutrients2.values.flatten().tolist()
iom_list

['Vitamin A',
 'Biotin',
 'Riboflavin',
 'Calcium',
 'Iron',
 'Potassium',
 'Vitamin C',
 'Choline',
 'Thiamin',
 'Chromium',
 'Magnesium',
 'Selenium',
 'Vitamin D',
 'Folate',
 'Vitamin B-6',
 'Copper',
 'Molybdenum',
 'Zinc',
 'Vitamin E',
 'Niacin',
 'Vitamin B-12',
 'Fluoride',
 'Manganese',
 nan,
 'Vitamin K',
 'Pantothenic Acid',
 nan,
 'Iodine',
 'Phosphorus',
 nan]

In [69]:
iom_list_3 = iom_list[np.logical_not(np.isnan(iom_list))]
iom_list_3

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [86]:
iom_list_2 = []
for item in iom_list:
    if item == np.nan:
        pass
    else:
        iom_list_2.append(item)
    
print(iom_list_2)        

['Vitamin A', 'Biotin', 'Riboflavin', 'Calcium', 'Iron', 'Potassium', 'Vitamin C', 'Choline', 'Thiamin', 'Chromium', 'Magnesium', 'Selenium', 'Vitamin D', 'Folate', 'Vitamin B-6', 'Copper', 'Molybdenum', 'Zinc', 'Vitamin E', 'Niacin', 'Vitamin B-12', 'Fluoride', 'Manganese', nan, 'Vitamin K', 'Pantothenic Acid', nan, 'Iodine', 'Phosphorus', nan]


In [71]:
iom_nutrients_list = []
for item in iom_list:
    print(item)
    print(type(item))
    if np.isnan(item):
        pass
    else:
        print(item)
        if re.search('B-', item):  # this gives us all the columns that include _100g, many of which are nutrients
            item.replace("B-", "B")
            iom_nutrients_list.append(item.lower())
        else:
            iom_nutrients_list.append(item.lower())
            
print(iom_nutrients_list)

Vitamin A
<class 'str'>


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''